In [2]:
import os
import functools

import cv2
import numpy as np

from tests.metaworld.envs.mujoco.sawyer_xyz.test_scripted_policies import ALL_ENVS, test_cases_latest_nonoise


mujoco here!
mujoco now here!
mujoco here!
mujoco now here!


/home/ykai/anaconda3/envs/metaGarage/lib/python3.9/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [3]:
def trajectory_generator(env, policy, act_noise_pct, res=(640, 480), camera='corner'):
    action_space_ptp = env.action_space.high - env.action_space.low

    env.reset()
    env.reset_model()
    o = env.reset()

    for _ in range(env.max_path_length):
        a = policy.get_action(o)
        a = np.random.normal(a, act_noise_pct * action_space_ptp)

        o, r, done, info = env.step(a)
        # Camera is one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
        # print(np.shape(env.sim.render(*res, mode='offscreen', camera_name=camera)[:,:,::-1]))
        yield r, done, info, env.sim.render(*res, mode='offscreen', camera_name=camera)[:,:,::-1]

# def writer_for(tag, fps, res):
#     if not os.path.exists('movies_my'):
#         os.mkdir('movies_my')
#     return cv2.VideoWriter(
#         f'movies_my/{tag}.avi',
#         cv2.VideoWriter_fourcc('M','J','P','G'),
#         fps,
#         res
#     )

def write_for_img(tag, img):
    if not os.path.exists('movies_my'):
        os.mkdir('movies_my')
    name = f'movies_my/{tag}.png'
    return cv2.imwrite(name, img)

In [4]:
!LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libGLEW.so:/usr/lib/nvidia/libGL.so


In [5]:
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/nvidia
!export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/usr/lib/nvidia-510
!export LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libGLEW.so

In [6]:
resolution = (1920, 1080)
camera = 'topview' # one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
camera = ['topview', 'corner', 'corner2', 'corner3', 'behindGripper', 'gripperPOV']
my_camera = ['my_corner3', 'my_topview1']
camera = my_camera
flip=True # if True, flips output image 180 degrees

config = [
    # env, action noise pct, cycles, quit on success
    ('assembly-v2', np.zeros(4), 1, True),
]
for camera in camera:
    if camera in ['corner', 'corner2', 'corner3', 'gripperPOV', 'my_corner3']:
        flip = False
    else:
        flip = True
    for env, noise, cycles, quit_on_success in config:
        tag = env + '-noise-' + np.array2string(noise, precision=2, separator=',', suppress_small=True)\
            + '-cycles-'+ str(cycles) +'-camera-'+ camera

        policy = functools.reduce(lambda a,b : a if a[0] == env else b, test_cases_latest_nonoise)[1]
        env = ALL_ENVS[env]()
        env._partially_observable = False
        env._freeze_rand_vec = False
        env._set_task_called = True

        # writer = writer_for(tag, env.metadata['video.frames_per_second'], resolution)
        # for _ in range(cycles):
        #     for r, done, info, img in trajectory_generator(env, policy, noise, resolution, camera):
        #         if flip: img = cv2.rotate(img, cv2.ROTATE_180)
        #         writer.write(img)
        #         if quit_on_success and info['success']:
        #             break
        # writer.release()
        
        for _ in range(cycles):
            for r, done, info, img in trajectory_generator(env, policy, noise, resolution, camera):
                print(np.shape(img))
                if flip: img = cv2.rotate(img, cv2.ROTATE_180)
                write_for_img(tag, img)
                break
            
                if quit_on_success and info['success']:
                    break
        
    

/home/ykai/anaconda3/envs/metaGarage/lib/python3.9/site-packages/gym/spaces/box.py:84: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/ssd_2/ssd2048/papers/metaworld/metaworld/policies/policy.py:41: UserWarning: Constant(s) may be too high. Environments clip response to [-1, 1]
  warnings.warn('Constant(s) may be too high. Environments clip response to [-1, 1]')


Found 3 GPUs for rendering. Using device 0.
(1080, 1920, 3)
(1080, 1920, 3)


In [7]:
def my_trajectory_generator(env, policy, act_noise_pct, res=(640, 480), camera='corner', depth = True):
    action_space_ptp = env.action_space.high - env.action_space.low
    print("E1")
    env.reset()
    env.reset_model()
    o = env.reset()

    for _ in range(env.max_path_length):
        a = policy.get_action(o)
        a = np.random.normal(a, act_noise_pct * action_space_ptp)

        o, r, done, info = env.step(a)
        # Camera is one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
        # print(np.shape(env.sim.render(*res, mode='offscreen', camera_name=camera)[:,:,::-1]))
        img, d = env.sim.render(*res, mode='offscreen', camera_name=camera,depth = depth)
        # print(depth)
        print("E")
        yield r, done, info, img, d
        
def write_for_img(tag, img):
    if not os.path.exists('movies_my'):
        os.mkdir('movies_my')
    name = f'movies_my/{tag}.png'
    return cv2.imwrite(name, img)

In [8]:
import numpy as np
import cv2


def bright_img(tag, img):
    if not os.path.exists('movies_my'):
        os.mkdir('movies_my')
    name = f'movies_my/{tag}.png'
    src = img
    val = 100
    array = np.full(src.shape, (val, val, val), dtype=np.uint8)

    add = cv2.add(src, array)
    return cv2.imwrite(name, add)

In [9]:
resolution = (1920, 1080)
camera = 'topview' # one of ['corner', 'topview', 'behindGripper', 'gripperPOV']
camera = ['topview', 'corner', 'corner2', 'corner3', 'behindGripper', 'gripperPOV']
my_camera = ['my_corner3', 'my_topview1']
camera = my_camera
flip=True # if True, flips output image 180 degrees

config = [
    # env, action noise pct, cycles, quit on success
    ('assembly-v2', np.zeros(4), 1, True),
]

for camera in camera:
    if camera in ['corner', 'corner2', 'corner3', 'gripperPOV', 'my_corner3']:
        flip = False
    else:
        flip = True
    for env, noise, cycles, quit_on_success in config:
        tag = env + '-noise-' + np.array2string(noise, precision=2, separator=',', suppress_small=True)\
            + '-cycles-'+ str(cycles) +'-camera-'+ camera

        policy = functools.reduce(lambda a,b : a if a[0] == env else b, test_cases_latest_nonoise)[1]
        env = ALL_ENVS[env]()
        env._partially_observable = False
        env._freeze_rand_vec = False
        env._set_task_called = True
        for _ in range(cycles):
            for r, done, info, img, d in my_trajectory_generator(env, policy, noise, resolution, camera, depth = True):

                if flip: img = cv2.rotate(img, cv2.ROTATE_180)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                write_for_img(tag, img)
                print(d)
                # d = d[:, np.newaxis] 
                # print(img.shape)
                # print(d.shape)
                # print(d*255)
                # d = bright_img(tag+'_depth', d)
                write_for_img(tag+'_depth', d)
                break
            
                if quit_on_success and info['success']:
                    break

E1
E
[[0.9997777  0.99977785 0.9997781  ... 0.9997956  0.9997954  0.99979514]
 [0.99977785 0.9997781  0.9997783  ... 0.9997958  0.99979556 0.9997953 ]
 [0.99977803 0.9997783  0.99977845 ... 0.999796   0.99979573 0.9997955 ]
 ...
 [0.9902057  0.9902049  0.9902041  ... 0.9886916  0.98869085 0.9886901 ]
 [0.9902004  0.9901996  0.99019885 ... 0.9886864  0.98868555 0.9886848 ]
 [0.99019516 0.9901944  0.99019355 ... 0.9886811  0.9886803  0.9886795 ]]
E1
E
[[0.99442875 0.99442875 0.99442875 ... 0.99442875 0.99442875 0.99442875]
 [0.99442875 0.99442875 0.99442875 ... 0.99442875 0.99442875 0.99442875]
 [0.99442875 0.99442875 0.99442875 ... 0.99442875 0.99442875 0.99442875]
 ...
 [0.99442875 0.99442875 0.99442875 ... 0.99442875 0.99442875 0.99442875]
 [0.99442875 0.99442875 0.99442875 ... 0.99442875 0.99442875 0.99442875]
 [0.99442875 0.99442875 0.99442875 ... 0.99442875 0.99442875 0.99442875]]


In [10]:
img[:, :, 0], img[:, :, 2] = img[:, :, 2], img[:, :, 0] 

In [16]:

name = f'movies_my/linemod_depth2.png'

img = cv2.imread(name)
# if img is None:
#     print('Image load failed')
#     sys.exit()
# cv2.imshow('lenna', img)
# cv2.waitKey()

# cv2.destroyAllWindows()
# color_img = cv2.imread(name, cv2.IMREAD_COLOR)
gray_img = cv2.imread(name)
print(gray_img.type)
print(np.shape(gray_img))
# # unch_img = cv2.imread(name, cv2.IMREAD_UNCHANGED)
# bright_img = cv2.add(gray_img, 100)
# # cv2.imshow('color_img',color_img)
# cv2.imshow('gray_img',gray_img)
# # cv2.imshow('unch_img',unch_img)

# cv2.waitKey()
# cv2.destroyAllWindows()

AttributeError: module 'numpy' has no attribute 'type'

In [12]:
print(np.max(img))
print(np.min(img))

15
0
